In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_array = cv2.imread("/content/drive/MyDrive/FER2013/train/0/Training_3908.jpg")

In [ ]:
img_array.shape

In [ ]:
plt.imshow(img_array)

In [ ]:
datadirectory = "/content/drive/MyDrive/FER2013/train/"

In [ ]:
classes = ["0","1","2","3","4","5","6"]   ## LIST OF CLASSES

In [ ]:
for category in classes:
  path = os.path.join(datadirectory , category)
  for img in os.listdir(path):
    img_array = cv2.imread(os.path.join(path, img))
    plt.imshow(cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB))
    plt.show()
    break
  break  

In [ ]:
img_size = 224    #CHANGED THE SIZE FROM 48 X 48 TO 224 X 224 WHICH IS ACCEPTED BY TRANSFER LEARNING MODEL 
new_array = cv2.resize(img_array,(img_size, img_size))
plt.imshow(cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
new_array.shape

NOW READING ALL THE IMAGES AND CONVERTING THEM TO ARRAY

In [ ]:
training_data = []
def create_training_Data():
  for category in classes:
    path = os.path.join(datadirectory , category)
    class_num = classes.index(category)   #Labeling or indexing
    for img in os.listdir(path):
       try:
            img_array = cv2.imread(os.path.join(path, img))
            new_array = cv2.resize(img_array,(img_size , img_size))
            training_data.append([new_array , class_num])
       except Exception as e:
            pass  

In [ ]:
create_training_Data()

In [ ]:
print(len(training_data))

In [ ]:
import random
random.shuffle(training_data)

In [ ]:
print(len((training_data)))

In [ ]:
x = []  # data / feature
y = [] #label / numbering

for features,label in training_data:
  x.append(features)
  y.append(label)

x = np.array(x).reshape(-1, img_size, img_size, 3)  ## coverting x array to 4 Dimension array. because we are USING deep learning architecture such as mobileNet which takes 4D images 


In [ ]:
type(x)

In [ ]:
x.shape


In [ ]:
X= np.array(x)
Y= np.array(y)

#DEEP LEARNING MODEL FOR TRAINING - TRANSFER LEARNING

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten


In [ ]:
model = tf.keras.applications.MobileNetV2()     ##mobileNet is a pre-trained model

In [ ]:
model.summary()

# TRANSFER LEARNING - TUNING , WEIGHTS WILL START FROM LAST CHECK POINT

In [ ]:
base_input = model.layers[0].input    ##input layer

In [ ]:
base_output = model.layers[-2].output

In [ ]:
base_output   ##last row was deleted, coz they were unneccessary

In [ ]:
final_output = layers.Dense(128)(base_output)   ## global pooling layer.
## adding new layer after the output of global pooling layer.
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(64)(final_output)
final_output = layers.Activation('relu')(final_output)
final_output = layers.Dense(7,activation='softmax')(final_output)   ## Classification layer
    ## output layer

In [ ]:
final_output

In [ ]:
new_model = keras.Model(inputs = base_input , outputs = final_output)

In [ ]:
new_model.summary()

In [ ]:
new_model.compile(optimizer='adam',loss='kullback_leibler_divergence',metrics = ['accuracy'])

In [31]:
history = new_model.fit(X, Y, batch_size=32, epochs=50, validation_split=0.1) 

Epoch 1/50
332/332 [==============================] - 87s 209ms/step - loss: 0.8566 - accuracy: 0.6385 - val_loss: 3.8493 - val_accuracy: 0.4414
Epoch 2/50
332/332 [==============================] - 69s 208ms/step - loss: 0.6828 - accuracy: 0.7206 - val_loss: 2.5898 - val_accuracy: 0.4160
Epoch 3/50
332/332 [==============================] - 69s 208ms/step - loss: 0.6006 - accuracy: 0.7569 - val_loss: 1.6735 - val_accuracy: 0.5518
Epoch 4/50
332/332 [==============================] - 69s 208ms/step - loss: 0.5445 - accuracy: 0.7771 - val_loss: 7.0916 - val_accuracy: 0.3039
Epoch 5/50
332/332 [==============================] - 69s 208ms/step - loss: 0.4889 - accuracy: 0.8064 - val_loss: 4.3051 - val_accuracy: 0.3650
Epoch 6/50
332/332 [==============================] - 69s 208ms/step - loss: 0.4551 - accuracy: 0.8190 - val_loss: 1.8540 - val_accuracy: 0.5424
Epoch 7/50
332/332 [==============================] - 69s 208ms/step - loss: 0.4145 - accuracy: 0.8329 - val_loss: 1.8683 - val_ac

In [32]:
new_model.save('/content/drive/MyDrive/final_model_95p07.h5')

In [33]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/final_model_95p07.h5')

In [ ]:
frame=cv2.imread("/content/drive/MyDrive/11.jpg")

In [ ]:
frame = np.array(frame)

In [ ]:
frame.shape

In [ ]:
plt.imshow(cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))

In [ ]:
faceCascade= cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [ ]:
gray =cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [ ]:
gray.shape


In [ ]:
faces =faceCascade.detectMultiScale(gray,1.1,4)  ##open cv code, find all possible faces in a photo, and can also detect multiple faces
for x,y,w,h in faces:
    roi_gray =gray[y:y+h,x:x+w]
    roi_color =frame[y:y+h, x:x+w]
    cv2.rectangle(frame, (x,y), (x+w,y+h), (225,0,0), 2)
    facess=faceCascade.detectMultiScale(roi_gray)
    if len(faces) ==0:
        print("Face not detected")
    else:
        for (ex,ey,ew,eh) in facess:
            face_roi =roi_color[ey: ey+eh, ex:ex +ew]


In [ ]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [ ]:
plt.imshow(cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB))

In [ ]:
final_image=cv2.resize(face_roi,(224,224))  ##face_roi is rgb
final_image =np.expand_dims(final_image,axis=0)  #need 4th dimensions

In [ ]:
Predictions=new_model.predict(final_image)

In [ ]:
Predictions[0]


In [ ]:
type(Predictions)

In [ ]:
Predictions.shape